In [1]:
import pandas as pd
from scipy.stats import zscore
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn import neural_network

In [2]:
data = pd.read_csv("../backend/dataset/indian_water_physicochemical_parameters.csv", encoding = 'unicode_escape')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
data['date'] = pd.to_datetime(data['date'])
data

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year,date
0,1393.0,DAMANGANGA AT D/S OF MADHUBAN,DAMAN & DIU,31.0,7.1,7.8,151.0,8.0,0.08,233.0,465.0,2012,2012-01-01
1,1150.0,DAMANGANGA AT KACHIGAON U\S AT GIDC WIER,GUJARAT,28.0,7.2,7.4,319.0,0.7,0.18,5.0,10.0,2012,2012-01-04
2,1865.0,RIVER DHADAR AT KOTHADA,GUJARAT,27.0,6.2,8.4,894.0,9.0,0.38,140.0,350.0,2012,2012-01-05
3,1149.0,KOLAK AT PATALIA BDG,GUJARAT,29.0,2.4,7.6,1500.0,6.2,1.31,3.0,13.0,2012,2012-01-06
4,1399.0,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOINS,GOA,31.0,5.9,7.2,52.0,2.1,0.08,233.0,90.0,2012,2012-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,2052.0,TUIRIAL UPPER CATCHMENT,MIZORAM,23.6,6.9,7.1,186.0,1.6,0.10,233.0,29.0,2014,2014-12-22
734,2053.0,TUIRIAL LOWER CATCHMENT,MIZORAM,23.5,6.8,7.1,201.0,1.4,0.20,233.0,86.0,2014,2014-12-24
735,1403.0,"GUMTI AT U/S SOUTH TRIPURA,TRIPURA",TRIPURA,27.3,6.2,7.8,165.0,2.1,0.00,282.0,340.0,2014,2014-12-25
736,1404.0,"GUMTI AT D/S SOUTH TRIPURA, TRIPURA",TRIPURA,29.1,6.4,7.8,158.0,4.1,0.50,458.0,500.0,2014,2014-12-26


In [9]:
import pandas as pd
df = data.copy()
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
specified_date = "2012-01-02"
if not isinstance(specified_date, pd.Timestamp):
    specified_date = pd.to_datetime(specified_date)

try:
    index_of_date = df.index.get_loc(specified_date)
    print("Index of specified date:", index_of_date)
except KeyError:
    print("Error: Specified date does not exist in the index.")


Error: Specified date does not exist in the index.


In [3]:

from aquasentinel import predict_wqi, get_wqc 
def wqi_gauge(specified_date):
    df = data.copy()
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    
    # Convert specified_date to datetime if it's not already
    if not isinstance(specified_date, pd.Timestamp):
        specified_date = pd.to_datetime(specified_date)
        
    try:
    # Get index of specified_date
        index_of_date = df.index.get_loc(specified_date)
        
        df = df.drop(columns=["STATION CODE", "LOCATIONS", "STATE", "Temp", "year"])
        
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors='coerce')
            
        df.fillna(df.median(), inplace=True)
        
        # Checking for outliers in the full dfset
        z_scores = df.apply(zscore)
        outliers_count = (z_scores.abs() > 3).sum()
        df_outliers = np.where(np.abs(z_scores) > 3)[0]
        df_filtered = df.drop(df.index[df_outliers])
        
        # Scale the df
        scaler = StandardScaler()
        X = df_filtered.values
        X_scaled = scaler.fit_transform(X)
        wqi = predict_wqi(X_scaled[index_of_date])
        wqc = get_wqc(wqi)
        return wqi, wqc
    
    except KeyError:
        print("Error: Specified date does not exist in the index.")
    


In [4]:
wqi_gauge("2012-01-01")

c:\Users\user\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MLPRegressor from version 1.3.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


(130.30465384652175, 'unsuitable')